# Interactive Chat with Steering (HuggingFace + Projection Capping)

This notebook provides an interactive chat interface using HuggingFace models with projection capping steering.

In [ ]:
import torch
import os
import json
import sys
from pathlib import Path

sys.path.append('.')
sys.path.append('..')

from utils.inference_utils import *
from utils.probing_utils import *
from utils.steering_utils import ActivationSteering

torch.set_float32_matmul_precision('high')

## Model Configuration

In [ ]:
MODEL_NAME = "Qwen/Qwen3-32B"
MODEL_READABLE = "Qwen 3 32B"
OUTPUT_DIR = "./results/qwen-3-32b/interactive_steering_hf"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Steering config path
STEERING_CONFIG_PATH = "/workspace/qwen-3-32b/capped/configs/contrast/role_trait_sliding_config.pt"

# Chat template options
CHAT_KWARGS = {'enable_thinking': False}  # Disable thinking for Qwen models

## Load Steering Config

In [ ]:
def load_steering_config(config_path):
    """Load steering config and return available experiments."""
    print(f"Loading steering config from {config_path}")
    cfg_data = torch.load(config_path, map_location='cpu')
    
    print(f"\nFound {len(cfg_data['vectors'])} vectors")
    print(f"Found {len(cfg_data['experiments'])} experiments")
    
    # Show first few experiment IDs
    print("\nFirst 10 experiments:")
    for i, exp in enumerate(cfg_data['experiments'][:10]):
        print(f"  {i}: {exp['id']}")
    
    return cfg_data

steering_config = load_steering_config(STEERING_CONFIG_PATH)

## Build Steering Context Manager

In [ ]:
def build_steering_context(model, cfg_data, experiment_id):
    """Build an ActivationSteering context manager from a config experiment.
    
    Only handles 'cap' (projection capping) interventions.
    Returns None if no capping interventions found.
    """
    # Find the experiment
    experiment = None
    if isinstance(experiment_id, int):
        experiment = cfg_data['experiments'][experiment_id]
    else:
        for exp in cfg_data['experiments']:
            if exp['id'] == experiment_id:
                experiment = exp
                break
    
    if experiment is None:
        raise ValueError(f"Experiment '{experiment_id}' not found")
    
    print(f"\nBuilding steering context for: {experiment['id']}")
    print(f"Interventions: {len(experiment['interventions'])}")
    
    # Collect capping interventions
    vectors = []
    cap_thresholds = []
    layer_indices = []
    
    for intervention in experiment['interventions']:
        # Only process capping interventions
        if 'cap' not in intervention:
            continue
            
        vector_name = intervention['vector']
        cap_value = float(intervention['cap'])
        
        # Get the vector and layer from the vectors dict
        vec_data = cfg_data['vectors'][vector_name]
        layer_idx = vec_data['layer']
        vector = vec_data['vector'].to(dtype=torch.float32)
        
        vectors.append(vector)
        cap_thresholds.append(cap_value)
        layer_indices.append(layer_idx)
    
    if not vectors:
        print("No capping interventions found in this experiment")
        return None
    
    affected_layers = sorted(set(layer_indices))
    print(f"Affecting layers: {affected_layers}")
    print(f"Cap thresholds: {cap_thresholds}")
    
    # Stack vectors and move to model device
    vectors_tensor = torch.stack(vectors)
    model_device = next(model.parameters()).device
    vectors_tensor = vectors_tensor.to(model_device)
    
    # Create ActivationSteering context manager
    return ActivationSteering(
        model=model,
        steering_vectors=vectors_tensor,
        layer_indices=layer_indices,
        intervention_type="capping",
        cap_thresholds=cap_thresholds,
        coefficients=[0.0] * len(vectors),  # Unused for capping but required
        positions="all"
    )

## Initialize Model

In [ ]:
print(f"Initializing {MODEL_READABLE}...")
model, tokenizer = load_model(MODEL_NAME)
print(f"✅ Model {MODEL_READABLE} loaded successfully!")

## Conversation State

In [ ]:
conversation_history = []
current_steering_context = None

## Chat Functions

In [ ]:
def chat_interactive(
    message, 
    steering_context=None,
    show_history=False,
    max_tokens=3000,
    temperature=0.7,
    use_steering=True
):
    """Interactive chat function with optional projection capping steering.
    
    Args:
        message: User message to send
        steering_context: ActivationSteering context to apply (uses current_steering_context if None)
        show_history: Whether to print conversation history
        max_tokens: Maximum tokens to generate
        temperature: Sampling temperature
        use_steering: Whether to apply steering at all
    """
    global conversation_history, current_steering_context
    
    # Add user message to history
    conversation_history.append({"role": "user", "content": message})
    
    # Format conversation for the model
    prompt = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=False,
        add_generation_prompt=True,
        **CHAT_KWARGS
    )
    
    # Use provided steering context or fall back to current
    active_context = steering_context if steering_context is not None else current_steering_context
    
    # Generate with or without steering
    if use_steering and active_context is not None:
        # Use steering context manager
        with active_context:
            response = generate_text(
                model,
                tokenizer,
                prompt,
                chat_format=False,  # Already formatted
                max_new_tokens=max_tokens,
                temperature=temperature
            )
    else:
        # No steering
        response = generate_text(
            model,
            tokenizer,
            prompt,
            chat_format=False,  # Already formatted
            max_new_tokens=max_tokens,
            temperature=temperature
        )
    
    # Add assistant response to history
    conversation_history.append({"role": "assistant", "content": response})
    
    # Print conversation
    print(f"👤 You: {message}")
    print(f"🤖 {MODEL_READABLE}: {response}")
    
    if show_history:
        print(f"\n📜 Conversation so far ({len(conversation_history)} turns):")
        for i, turn in enumerate(conversation_history):
            role_emoji = "👤" if turn["role"] == "user" else "🤖" 
            content_preview = turn['content'][:100] + "..." if len(turn['content']) > 100 else turn['content']
            print(f"  {i+1}. {role_emoji} {content_preview}")
    
    return response

## Helper Functions

In [ ]:
def save_conversation(filename):
    """Save the current conversation to a file."""
    if not conversation_history:
        print("No conversation to save!")
        return
    
    conversation_data = {
        "model": MODEL_NAME,
        "turns": len(conversation_history),
        "conversation": conversation_history,
        "steering_active": current_steering_context is not None
    }
    
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, 'w') as f:
        json.dump(conversation_data, f, indent=2)
    
    print(f"💾 Conversation saved to: {filepath}")
    return filepath


def reset_conversation():
    """Reset the conversation history."""
    global conversation_history
    conversation_history = []
    print("🔄 Conversation history cleared!")


def delete_last_turn():
    """Delete the last turn from the conversation history."""
    global conversation_history
    if conversation_history:
        # Remove last two entries (user + assistant)
        if len(conversation_history) >= 2:
            conversation_history = conversation_history[:-2]
            print("🔄 Last turn deleted!")
        else:
            conversation_history = []
            print("🔄 Conversation cleared (was incomplete turn)!")
    else:
        print("No conversation to delete!")


def set_steering(experiment_id):
    """Set the current steering context by experiment ID or index."""
    global current_steering_context
    current_steering_context = build_steering_context(model, steering_config, experiment_id)
    if current_steering_context:
        print(f"✅ Steering set to experiment: {experiment_id}")
    else:
        print(f"⚠️  No capping interventions found for experiment: {experiment_id}")


def clear_steering():
    """Clear the current steering."""
    global current_steering_context
    current_steering_context = None
    print("✅ Steering cleared")


def list_experiments(start=0, end=20):
    """List available experiments."""
    experiments = steering_config['experiments'][start:end]
    print(f"\nExperiments {start} to {min(end, len(steering_config['experiments']))}:")
    for i, exp in enumerate(experiments, start=start):
        # Count capping interventions
        cap_count = sum(1 for interv in exp['interventions'] if 'cap' in interv)
        print(f"  {i}: {exp['id']} ({cap_count} capping interventions)")
    print(f"\nTotal experiments: {len(steering_config['experiments'])}")

## Example Usage

In [ ]:
# List available experiments
list_experiments(40, 50)

In [ ]:
# Set steering to a specific experiment
set_steering("layers_54:58-p0.25")

In [ ]:
# Chat with steering active
chat_interactive("Hello! How are you today?")

In [ ]:
# Continue the conversation
chat_interactive("What's your perspective on artificial consciousness?")

In [ ]:
# Compare with no steering
reset_conversation()
chat_interactive("Hello! How are you today?", use_steering=False)

In [ ]:
# Try a different steering experiment
reset_conversation()
set_steering(45)  # Different experiment
chat_interactive("Hello! How are you today?")

In [ ]:
# Save the conversation
save_conversation("steering_experiment_45.json")

In [ ]:
# Clear steering and continue
clear_steering()
chat_interactive("What do you think about AI safety?")

## One-off Steering

You can also apply steering for just a single message without changing the global state:

In [ ]:
# Build a one-off steering context
temp_context = build_steering_context(model, steering_config, 10)

# Use it for just this message
chat_interactive(
    "Tell me about yourself.",
    steering_context=temp_context
)